## To Do

- what is a good chunk size?
- how can I verify a better performance?
- check accuracy of results
- check computation time on cluster

In [1]:
from toad import TOAD
import numpy as np
import xarray as xr
from toad.shifts_detection.methods import ASDETECT as ASDETECT
#import dask

fp = "tutorials/test_data/garbe_2020_antarctica.nc"
#fp = "tutorials/test_data/global_mean_summer_tas.nc"
var = "thk"

data = xr.open_dataset(fp)
spatial_dims = list(data[var].dims)
spatial_dims.remove("time")

c = 5
c_dict = {dim: c for dim in spatial_dims}
c_dict["time"] = 3
data = data.coarsen(**c_dict,
                    boundary="trim").reduce(np.mean)

print(f"Dimensions after coarsening:\n{data.sizes}")

cs = None
print(data[var].chunk({'x': cs, 'y': cs}).data.nbytes / 1e6, "MB")


Dimensions after coarsening:
Frozen({'time': 116, 'y': 38, 'x': 38})
0.670016 MB


In [2]:
from dask.distributed import Client

client = Client(n_workers=5)
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 15.29 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43491,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:34241,Total threads: 2
Dashboard: http://127.0.0.1:42651/status,Memory: 3.06 GiB
Nanny: tcp://127.0.0.1:45665,


In [5]:
#dask.config.set(scheduler='threads')

td_new = TOAD(data)
td_new.compute_shifts(var,
                  method=ASDETECT(),
                  overwrite=True,
                  chunk_size=10,
                  dask_compute=True)

[                                        ] | 0% Completed | 169.48 us

[########################################] | 100% Completed | 1.67 sms


## Chunk Size

In [6]:
import time

# Define the chunk sizes to test
chunk_sizes = [None, 5, 20, 30, 40, 50]
sample_size = 5

# Run tests
results = [0] * len(chunk_sizes)

print("Benchmarking chunk sizes...\n")
for i in range(len(chunk_sizes)):
    size = chunk_sizes[i]
    for j in range(sample_size):
        # get test data
        td = TOAD(data)

        # Time the execution
        lazy_shifts = td_new.compute_shifts(var,
                                            method=ASDETECT(),
                                            overwrite=True,
                                            return_results_directly=True,
                                            chunk_size=size,
                                            dask_compute=False)
        
        start_time = time.time()
        _ = lazy_shifts.compute()
        elapsed = time.time() - start_time

        results[i] += elapsed
    results[i] /= sample_size
    print(f"Chunk size {size}x{size}: {results[i]:.2f} seconds")

Benchmarking chunk sizes...

Chunk size NonexNone: 1.29 seconds
Chunk size 5x5: 1.42 seconds
Chunk size 20x20: 1.41 seconds
Chunk size 30x30: 1.33 seconds
Chunk size 40x40: 1.43 seconds
Chunk size 50x50: 1.33 seconds


## Artificial Dataset

In [13]:
import dask.array as da
import xarray as xr
import numpy as np

# Parameters
shape = (50, 50000, 50000)
chunks = (1, 2500, 2500)

# Create Dask array lazily
data_dask = da.random.random(shape, chunks=chunks)

# Wrap in xarray
time = np.arange(shape[0])
lat = np.linspace(-90, 90, shape[1])
lon = np.linspace(-180, 180, shape[2])

data = xr.DataArray(
    data_dask,
    dims=["time", "x", "y"],
    coords={"time": time, "x": lat, "y": lon}
)

sizes = [np.prod(c) * data.dtype.itemsize for c in zip(*data.chunks)]
print("Chunk sizes (MB):", [float(s) / 1e6 for s in sizes[:]])  # just showing first 5


Chunk sizes (MB): [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]


In [ ]:
import xarray as xr
import numpy as np

# Define large shape (e.g., 1000 time steps, 1000x1000 spatial grid)
shape = (500, 1000, 1000)  # (time, lat, lon)

# Create coordinate values
time = np.arange(shape[0])
lat = np.linspace(-90, 90, shape[1])
lon = np.linspace(-180, 180, shape[2])

# Create synthetic data using Dask
data = xr.DataArray(
    np.random.rand(*shape),
    dims=["time", "lat", "lon"],
    coords={"time": time, "lat": lat, "lon": lon}
)
td_art = TOAD(data) 

print(td_art.data.dims)


('time', 'lat', 'lon')


: 

In [ ]:
import time

# Define the chunk sizes to test
chunk_sizes = [50, 100, 500]

# Run tests
results = []

print("Benchmarking chunk sizes...\n")
for size in chunk_sizes:
    # get test data
    td = TOAD(data)

    # Time the execution
    start_time = time.time()
    td_art.compute_shifts(var,
                    method=ASDETECT(),
                    overwrite=True,
                    chunk_size=size,)
    elapsed = time.time() - start_time

    results.append((size, elapsed))
    print(f"Chunk size {size}x{size}: {elapsed:.2f} seconds")

# Summary
print("\nSummary:")
for size, elapsed in results:
    print(f"Chunk size {size}x{size}: {elapsed:.2f} s")


Benchmarking chunk sizes...

